# **DataExtraction help of python and its libraries**.

# **Objective:**
The objective of this assignment is to extract textual data articles from the given URL and perform text analysis to compute variables that are explained below.


For extraction of data and crawling we need to import the necessary libraries like pandas,textstat,Beautifulsoup etc for text manpulation.

In [1]:
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from textstat import flesch_reading_ease, syllable_count

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Data Extraction**:
Input.xlsx
For each of the articles, given in the input.xlsx file, extract the article text and save the extracted article in a text file with URL_ID as its file name.


In [3]:
# Read the input Excel file containing URLs
input_file = "/content/Input.xlsx"
input_df = pd.read_excel(input_file)

In this process we extract the data to sepearte index and rows.

In [4]:
for index, row in input_df.iterrows():
    url_id = row['URL_ID']  # Assuming 'URL_ID' is the column name in the input file
    url = row['URL']

To create a Function to extract the data from the given url

In [5]:
# Function to extract article text from a given URL
def extract_article_text(url):
    try:
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if request was successful
        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract article text
            article_text = ""
            # Assuming article text is within <p> tags or specific class/id
            paragraphs = soup.find_all('p')
            for paragraph in paragraphs:
                article_text += paragraph.get_text() + "\n"

            return article_text
        else:
            print(f"Failed to retrieve data from URL: {url}")
            return None
    except Exception as e:
        print(f"Error occurred while processing URL: {url}\nError: {e}")
        return None

article_text = extract_article_text(url)

# **Variables:**
1.	POSITIVE SCORE
2.	NEGATIVE SCORE
3.	POLARITY SCORE
4.	SUBJECTIVITY SCORE
5.	AVG SENTENCE LENGTH
6.	PERCENTAGE OF COMPLEX WORDS
7.	FOG INDEX
8.	AVG NUMBER OF WORDS PER SENTENCE
9.	COMPLEX WORD COUNT
10.	WORD COUNT
11.	SYLLABLE PER WORD
12.	PERSONAL PRONOUNS
13.	AVG WORD LENGTH
Created the formula through help of text document.


In [6]:
# Function to perform textual analysis
def perform_textual_analysis(article_text):
    # Tokenize sentences
    sentences = sent_tokenize(article_text)

    # Tokenize words
    words = word_tokenize(article_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate positive and negative scores using TextBlob
    blob = TextBlob(article_text)
    sentiment_scores = blob.sentiment

    positive_score = sentiment_scores.polarity if sentiment_scores.polarity > 0 else 0
    negative_score = -sentiment_scores.polarity if sentiment_scores.polarity < 0 else 0

    # Calculate polarity and subjectivity scores
    polarity_score = sentiment_scores.polarity
    subjectivity_score = sentiment_scores.subjectivity

    # Calculate average sentence length
    avg_sentence_length = sum(len(word_tokenize(sentence)) for sentence in sentences) / len(sentences)

    # Calculate percentage of complex words
    complex_word_count = sum(1 for word in filtered_words if syllable_count(word) > 2)
    percentage_complex_words = (complex_word_count / len(filtered_words)) * 100 if len(filtered_words) > 0 else 0

    # Calculate FOG index
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Calculate average number of words per sentence
    avg_words_per_sentence = len(filtered_words) / len(sentences) if len(sentences) > 0 else 0

    # Calculate average word length
    avg_word_length = sum(len(word) for word in filtered_words) / len(filtered_words) if len(filtered_words) > 0 else 0

    # Calculate syllables per word
    syllables_per_word = sum(syllable_count(word) for word in filtered_words) / len(filtered_words) if len(filtered_words) > 0 else 0

    # Count personal pronouns
    personal_pronouns = sum(1 for word in filtered_words if word.lower() in ['i', 'me', 'my', 'mine', 'myself', 'you', 'your', 'yours', 'yourself', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'we', 'us', 'our', 'ours', 'ourselves', 'they', 'them', 'their', 'theirs', 'themselves'])

    return positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count, len(filtered_words), syllables_per_word, personal_pronouns, avg_word_length

In [9]:
 if article_text:
        # Perform textual analysis
        (positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length,
         percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count,
         word_count, syllables_per_word, personal_pronouns, avg_word_length) = perform_textual_analysis(article_text)

# **Generate the output col dataset**.

In [10]:
output_columns = ['URL_ID', 'Positive Score', 'Negative Score', 'Polarity Score', 'Subjectivity Score', 'Avg Sentence Length', 'Percentage of Complex Words', 'FOG Index', 'Avg Words Per Sentence', 'Complex Word Count', 'Word Count', 'Syllables Per Word', 'Personal Pronouns', 'Avg Word Length']
output_df = pd.DataFrame(columns=output_columns)


# **Append the extract data to the output dataframe.**

In [12]:
 # Append data to output DataFrame
output_df = output_df.append({
            'URL_ID': url_id,
            'Positive Score': positive_score,
            'Negative Score': negative_score,
            'Polarity Score': polarity_score,
            'Subjectivity Score': subjectivity_score,
            'Avg Sentence Length': avg_sentence_length,
            'Percentage of Complex Words': percentage_complex_words,
            'FOG Index': fog_index,
            'Avg Words Per Sentence': avg_words_per_sentence,
            'Complex Word Count': complex_word_count,
            'Word Count': word_count,
            'Syllables Per Word': syllables_per_word,
            'Personal Pronouns': personal_pronouns,
            'Avg Word Length': avg_word_length
        }, ignore_index=True)


<ipython-input-12-213c51be77ce>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({


# **Convert the output from the dataset into the output file**

In [13]:
output_file = "Output.xlsx"
output_df.to_excel(output_file, index=False)
print("Output saved to:", output_file)


Output saved to: Output.xlsx


# **Saved The Outfile and Extract it.**

In [14]:
data=pd.read_excel("/content/Output.xlsx")

In [15]:
data.head()

,URL_ID,Positive Score,Negative Score,Polarity Score,Subjectivity Score,Avg Sentence Length,Percentage of Complex Words,FOG Index,Avg Words Per Sentence,Complex Word Count,Word Count,Syllables Per Word,Personal Pronouns,Avg Word Length
0,blackassign0100,0.086026,0,0.086026,0.416633,37.647059,25.524044,25.268441,23.852941,207,811,1.733662,1,5.885327
